# Inference on custom images

## Download code and install dependencies

In [1]:
!git clone https://github.com/j-min/CLIP-Caption-Reward

fatal: destination path 'CLIP-Caption-Reward' already exists and is not an empty directory.


In [2]:
cd CLIP-Caption-Reward

/kaggle/working/CLIP-Caption-Reward


In [3]:
!pip install -r requirements.txt
!pip uninstall -y  torchtext # to bypass pt-lightning issue (https://github.com/PyTorchLightning/pytorch-lightning/issues/6415)
!pip install -e .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.4/510.4 kB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 2.2.2
    Uninstalling pytorch-lightning-2.2.2:
      Successfully uninstalled pytorch-lightning-2.2.2
Found existing installation: torchtext 0.16.2
Uninstalling torchtext-0.16.2:
  Successfully uninstalled torchtext-0.16.2
Obtaining file:///kaggle/working/CLIP-Caption-Reward
  Preparing metadata (setup.py) ... done
  Running setup.py develop for captioning


# Import packages

In [4]:
import torch
import torch.nn as nn

import numpy as np

import json

import captioning.utils.opts as opts
import captioning.models as models
import captioning.utils.misc as utils

import pytorch_lightning as pl

# Checkpoint class
class ModelCheckpoint(pl.callbacks.ModelCheckpoint):

    def on_keyboard_interrupt(self, trainer, pl_module):
        # Save model when keyboard interrupt
        filepath = os.path.join(self.dirpath, self.prefix + 'interrupt.ckpt')
        self._save_model(filepath)

meshed-memory-transformer not installed; please run `pip install git+https://github.com/ruotianluo/meshed-memory-transformer.git`


2024-04-24 04:03:59.736942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 04:03:59.737069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 04:03:59.986863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
!pip install git+https://github.com/ruotianluo/meshed-memory-transformer.git

  Cloning https://github.com/ruotianluo/meshed-memory-transformer.git to /tmp/pip-req-build-t9fn5tx8
  Running command git clone --filter=blob:none --quiet https://github.com/ruotianluo/meshed-memory-transformer.git /tmp/pip-req-build-t9fn5tx8
  Resolved https://github.com/ruotianluo/meshed-memory-transformer.git to commit f74657320a947ad1d9868de6a6c757dbac341eda
  Preparing metadata (setup.py) ... done
  Created wheel for meshed_memory_transformer: filename=meshed_memory_transformer-0.0.1-py3-none-any.whl size=39521 sha256=16e920adbc0f9e543f0a59dd9c9ca690d8c87196ea220a558772b62f3b41da3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-haye34eb/wheels/98/10/2e/74211626a660b85e2332e7884db384c5aca1894d57fb1eddc2
Successfully built meshed_memory_transformer


# Device and model configurations

In [78]:
device = 'cuda' #@param ["cuda", "cpu"] {allow-input: true}

reward = 'mle' #@param ["mle", "cider", "clips", "cider_clips", "clips_grammar"] {allow-input: true}

if reward == 'mle':
    cfg = f'./configs/phase1/clipRN50_{reward}.yml'
else:
    cfg = f'./configs/phase2/clipRN50_{reward}.yml'

print("Loading cfg from", cfg)

opt = opts.parse_opt(parse=False, cfg=cfg)

Loading cfg from ./configs/phase1/clipRN50_mle.yml


In [79]:
# print(opt)

# Download pretrained checkpoint

In [7]:
!pip install gdown

In [8]:
import gdown

In [80]:
if reward == "mle":
  url = "https://drive.google.com/drive/folders/1hfHWDn5iXsdjB63E5zdZBAoRLWHQC3LD"
elif reward == "cider":
  url = "https://drive.google.com/drive/folders/1MnSmCd8HFnBvQq_4K-q4vsVkzEw0OIOs"
elif reward == "clips":
  url = "https://drive.google.com/drive/folders/1toceycN-qilHsbYjKalBLtHJck1acQVe"
elif reward == "cider_clips":
  url = "https://drive.google.com/drive/folders/1toceycN-qilHsbYjKalBLtHJck1acQVe"
elif reward == "clips_grammar":
  url = "https://drive.google.com/drive/folders/1nSX9aS7pPK4-OTHYtsUD_uEkwIQVIV7W"
gdown.download_folder(url, quiet=True, use_cookies=False, output="save/")

['save/clipRN50_mle/clipRN50_mle-last.ckpt']

# Load vocabulary

In [63]:
# url for the videos
# url = "https://drive.google.com/drive/folders/1PfKgVYgj8kzOCctI67deAa3Aozpn1ZeL"
# gdown.download_folder(url, quiet=True, use_cookies=False, output="data/")

url = "https://drive.google.com/uc?id=1HNRE1MYO9wxmtMHLC8zURraoNFu157Dp"
gdown.download(url, quiet=True, use_cookies=False, output="data/")

'data/cocotalk.json'

In [14]:
# !rm data/view\?usp\=drive_link

In [81]:
dict_json = json.load(open('./data/cocotalk.json'))
print(dict_json.keys())

ix_to_word = dict_json['ix_to_word']
vocab_size = len(ix_to_word)
print('vocab size:', vocab_size)

seq_length = 1

opt.vocab_size = vocab_size
opt.seq_length = seq_length

dict_keys(['ix_to_word', 'images'])
vocab size: 9487


# Load Model checkpoint

In [82]:
opt.batch_size = 1
opt.vocab = ix_to_word
# opt.use_grammar = False

model = models.setup(opt)
del opt.vocab

ckpt_path = opt.checkpoint_path + '-last.ckpt'

print("Loading checkpoint from", ckpt_path)
raw_state_dict = torch.load(
    ckpt_path,
    map_location=device)

strict = True

state_dict = raw_state_dict['state_dict']

if '_vocab' in state_dict:
    model.vocab = utils.deserialize(state_dict['_vocab'])
    del state_dict['_vocab']
elif strict:
    raise KeyError
if '_opt' in state_dict:
    saved_model_opt = utils.deserialize(state_dict['_opt'])
    del state_dict['_opt']
    # Make sure the saved opt is compatible with the curren topt
    need_be_same = ["caption_model",
                    "rnn_type", "rnn_size", "num_layers"]
    for checkme in need_be_same:
        if getattr(saved_model_opt, checkme) in ['updown', 'topdown'] and \
                getattr(opt, checkme) in ['updown', 'topdown']:
            continue
        assert getattr(saved_model_opt, checkme) == getattr(
            opt, checkme), "Command line argument and saved model disagree on '%s' " % checkme
elif strict:
    raise KeyError
res = model.load_state_dict(state_dict, strict)
print(res)

model = model.to(device)
model.eval();

Loading checkpoint from save/clipRN50_mle/clipRN50_mle-last.ckpt
<All keys matched successfully>


# Load CLIP image encoder

In [83]:
import clip
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
from timm.models.vision_transformer import resize_pos_embed

clip_model, clip_transform = clip.load("RN50", jit=False, device=device)

preprocess = Compose([
    Resize((448, 448), interpolation=Image.BICUBIC),
    CenterCrop((448, 448)),
    ToTensor()
])

image_mean = torch.Tensor([0.48145466, 0.4578275, 0.40821073]).to(device).reshape(3, 1, 1)
image_std = torch.Tensor([0.26862954, 0.26130258, 0.27577711]).to(device).reshape(3, 1, 1)

num_patches = 196 #600 * 1000 // 32 // 32
pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, clip_model.visual.attnpool.positional_embedding.shape[-1],  device=device),)
pos_embed.weight = resize_pos_embed(clip_model.visual.attnpool.positional_embedding.unsqueeze(0), pos_embed)
clip_model.visual.attnpool.positional_embedding = pos_embed

# Extract Visual feature

In [27]:
# !pip install ipython

In [14]:
img_path = './assets/COCO_val2014_000000462565.jpeg'
# from IPython.display import Image as show_image
# show_image(img_path)

In [15]:
with torch.no_grad():
    image = preprocess(Image.open( img_path ).convert("RGB"))
    image = torch.tensor(np.stack([image])).to(device)
    image -= image_mean
    image /= image_std

    tmp_att, tmp_fc = clip_model.encode_image(image)
    tmp_att = tmp_att[0].permute(1, 2, 0)
    tmp_fc = tmp_fc[0]

    att_feat = tmp_att
    fc_feat = tmp_fc

# Generate caption

In [16]:
# Inference configurations
eval_kwargs = {}
eval_kwargs.update(vars(opt))

verbose = eval_kwargs.get('verbose', True)
verbose_beam = eval_kwargs.get('verbose_beam', 0)
verbose_loss = eval_kwargs.get('verbose_loss', 1)

# dataset = eval_kwargs.get('dataset', 'coco')
beam_size = eval_kwargs.get('beam_size', 1)
sample_n = eval_kwargs.get('sample_n', 1)
remove_bad_endings = eval_kwargs.get('remove_bad_endings', 0)

with torch.no_grad():
    fc_feats = torch.zeros((1,0)).to(device)
    att_feats = att_feat.view(1, 196, 2048).float().to(device)
    att_masks = None

    # forward the model to also get generated samples for each image
    # Only leave one feature for each image, in case duplicate sample
    tmp_eval_kwargs = eval_kwargs.copy()
    tmp_eval_kwargs.update({'sample_n': 1})
    seq, seq_logprobs = model(
        fc_feats, att_feats, att_masks, opt=tmp_eval_kwargs, mode='sample')
    seq = seq.data

    sents = utils.decode_sequence(model.vocab, seq)

print(sents)

['a group of people riding bikes down a city street']


In [17]:
import cv2
import matplotlib.pyplot as plt

In [34]:
def nograd(frame):
    with torch.no_grad():
        pilimg = Image.fromarray(frame)
        image = preprocess(pilimg).convert("RGB")
        image = torch.tensor(np.stack([image])).to(device)
        image -= image_mean
        image /= image_std

        tmp_att, tmp_fc = clip_model.encode_image(image)
        tmp_att = tmp_att[0].permute(1, 2, 0)
        tmp_fc = tmp_fc[0]

        att_feat = tmp_att
        fc_feat = tmp_fc
    return image

In [42]:
from torchvision.transforms.functional import to_pil_image


In [51]:
def nograd(frame):
    with torch.no_grad():
        # Convert numpy array to PIL Image
        pil_image = Image.fromarray(frame)
        
        # Apply preprocessing
        image = preprocess(pil_image)
        
        # Convert the preprocessed image back to PIL Image
        image_pil = to_pil_image(image)
        
        # Convert PIL Image to tensor
        image_tensor = ToTensor()(image_pil)
        
        image_tensor = torch.tensor(np.stack([image_tensor])).to(device)
        image_tensor -= image_mean
        image_tensor /= image_std

        tmp_att, tmp_fc = clip_model.encode_image(image_tensor)
        tmp_att = tmp_att[0].permute(1, 2, 0)
        tmp_fc = tmp_fc[0]

        att_feat = tmp_att
        fc_feat = tmp_fc
    return image_tensor


In [52]:
def gen_captions():
    # Inference configurations
    eval_kwargs = {}
    eval_kwargs.update(vars(opt))

    verbose = eval_kwargs.get('verbose', True)
    verbose_beam = eval_kwargs.get('verbose_beam', 0)
    verbose_loss = eval_kwargs.get('verbose_loss', 1)

    # dataset = eval_kwargs.get('dataset', 'coco')
    beam_size = eval_kwargs.get('beam_size', 1)
    sample_n = eval_kwargs.get('sample_n', 1)
    remove_bad_endings = eval_kwargs.get('remove_bad_endings', 0)

    with torch.no_grad():
        fc_feats = torch.zeros((1,0)).to(device)
        att_feats = att_feat.view(1, 196, 2048).float().to(device)
        att_masks = None

        # forward the model to also get generated samples for each image
        # Only leave one feature for each image, in case duplicate sample
        tmp_eval_kwargs = eval_kwargs.copy()
        tmp_eval_kwargs.update({'sample_n': 1})
        seq, seq_logprobs = model(
            fc_feats, att_feats, att_masks, opt=tmp_eval_kwargs, mode='sample')
        seq = seq.data

        sents = utils.decode_sequence(model.vocab, seq)

    return sents

In [97]:
#Extracting Frames form
res = []
def extract_frames_per_second(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get the FPS of the video
    print("fps:", fps)
    frame_count = 0
    extracted_count = 0
    ii = 0
    # if not os.path.exists(output_folder):
    #     os.makedirs(output_folder)
    while True:
        ii += 1
        if ii%50==0:
            print("ii:", ii)
        success, frame = cap.read()
#         print("fr type:", type(frame))
        if not success:
            break
        if frame_count % fps == 0:
            # cv2.imwrite(f'{output_folder}/frame_{extracted_count:04d}.png', frame)
            with torch.no_grad():
                # Convert numpy array to PIL Image
                pil_image = Image.fromarray(frame)

                # Apply preprocessing
                image = preprocess(pil_image)

                # Convert the preprocessed image back to PIL Image
                image_pil = to_pil_image(image)

                # Convert PIL Image to tensor
                image_tensor = ToTensor()(image_pil)

                image_tensor = torch.tensor(np.stack([image_tensor])).to(device)
                image_tensor -= image_mean
                image_tensor /= image_std

                tmp_att, tmp_fc = clip_model.encode_image(image_tensor)
                tmp_att = tmp_att[0].permute(1, 2, 0)
                tmp_fc = tmp_fc[0]

                att_feat = tmp_att
                fc_feat = tmp_fc
#             return image_tensor
            
            eval_kwargs = {}
            eval_kwargs.update(vars(opt))

            verbose = eval_kwargs.get('verbose', True)
            verbose_beam = eval_kwargs.get('verbose_beam', 0)
            verbose_loss = eval_kwargs.get('verbose_loss', 1)

            # dataset = eval_kwargs.get('dataset', 'coco')
            beam_size = eval_kwargs.get('beam_size', 1)
            sample_n = eval_kwargs.get('sample_n', 1)
            remove_bad_endings = eval_kwargs.get('remove_bad_endings', 0)

            with torch.no_grad():
                fc_feats = torch.zeros((1,0)).to(device)
                att_feats = att_feat.view(1, 196, 2048).float().to(device)
                att_masks = None

                # forward the model to also get generated samples for each image
                # Only leave one feature for each image, in case duplicate sample
                tmp_eval_kwargs = eval_kwargs.copy()
                tmp_eval_kwargs.update({'sample_n': 1})
                seq, seq_logprobs = model(
                    fc_feats, att_feats, att_masks, opt=tmp_eval_kwargs, mode='sample')
                seq = seq.data

                sents = utils.decode_sequence(model.vocab, seq)

            res.append(sents[0])
            
#             out = gen_captions()
#             res.append(out)
            extracted_count += 1

        frame_count += 1
    print(*res, sep='\n')
    cap.release()
    print(f"Extracted {extracted_count} frames (1 frame per second).")


In [99]:
extract_frames_per_second("./data/AD/00000n262-30000157739826_050_mezzCrop-high.mp4","")

fps: 24
ii: 50
ii: 100
ii: 150
ii: 200
ii: 250
ii: 300
ii: 350
ii: 400
ii: 450
ii: 500
ii: 550
ii: 600
ii: 650
ii: 700
ii: 750
ii: 800
ii: 850
ii: 900
ii: 950
ii: 1000
ii: 1050
ii: 1100
ii: 1150
ii: 1200
ii: 1250
ii: 1300
ii: 1350
ii: 1400
ii: 1450
a dark night with a dark background and a dark background
a woman in a dress is posing for a picture
a woman with a hat and a scarf on
a man brushing his teeth in front of a tree
a black and white photo of a tree with a branch in the background
a man laying on a bed with a dog
a woman holding a stuffed animal in her hand
a sign that says UNK on it is in the sky
a woman in a dress and a tie is smiling
a woman in a hat and scarf sitting in front of a tree
a woman riding on the back of a motorcycle
a woman riding on the back of a motorcycle
a boat traveling down a wet road near a beach
a black and white photo of a sign and a building
a person riding a motorcycle on a beach
a woman is eating food with a fork
a black and white photo of a bird fly

In [101]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.7 MB/s eta 0:00:00a 0:00:01m


In [105]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
sentence_embeddings = model.encode('This is an example')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [106]:
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of strings.",
    "The quick brown fox jumps over the lazy dog.",
]

# Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")